In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# ---------------------------
# 1. Load MNIST dataset
# ---------------------------

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # normalize grayscale images
])

trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# ---------------------------
# 2. Define CNN Model
# ---------------------------

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolution: input 1 channel (grayscale), output 32 filters, kernel size 3x3
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  
        self.pool = nn.MaxPool2d(2, 2)  # downsample by factor of 2
        # Second convolution: input 32, output 64 filters
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  
        self.fc2 = nn.Linear(128, 10)  # 10 classes (digits 0–9)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(torch.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = x.view(-1, 64 * 7 * 7)  # flatten
        x = torch.relu(self.fc1(x))  
        x = self.fc2(x)  
        return x

# ---------------------------
# 3. Training Setup
# ---------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()  # good for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ---------------------------
# 4. Training Loop
# ---------------------------

epochs = 3
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")

print("Training Finished ✅")

6.9%


KeyboardInterrupt: 